## 📥 Step 1: Load Excel Data
We load the cleaned datasets:
- `Cleaned_PE.xlsx`: PE firm metadata  
- `outputcheckcombined.xlsx`: Target company data  
- `PE_Transactions.xlsx`: Known PE-company acquisition pairs


In [2]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict

# Load the Excel files
pe_df = pd.read_excel("../newdata/Cleaned_PE.xlsx")
company_df = pd.read_excel("../newdata/outputcheckcombined.xlsx")
transactions_df = pd.read_excel("../newdata/PE_Transactions.xlsx")


c:\Users\Samir\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 🧪 Step 2: Normalize Sectors & Compute Embeddings
We lowercase and clean the sector descriptions and compute SBERT embeddings using the `all-MiniLM-L6-v2` model.


In [3]:
# Initialize SBERT
sbert = SentenceTransformer('all-MiniLM-L6-v2')

# Normalize text
company_df['normalized_sectors'] = company_df['1º_characterization'].fillna('').str.lower()
pe_df['normalized_sectors'] = pe_df['Sectors'].fillna('').str.lower()

# Compute embeddings
company_df['sector_embeddings'] = company_df['normalized_sectors'].apply(lambda x: sbert.encode(x))
pe_df['sector_embeddings'] = pe_df['normalized_sectors'].apply(lambda x: sbert.encode(x))


## 🔄 Step 3: Cross Join and Compute Cosine Similarity
We generate all combinations of PE–Company pairs and measure cosine similarity between their sector embeddings.


In [4]:
# Cross join
company_df['key'] = 1
pe_df['key'] = 1
pairs_df = company_df.merge(pe_df, on='key').drop('key', axis=1)

# Sector similarity using cosine
pairs_df['sector_similarity_score'] = pairs_df.apply(
    lambda row: cosine_similarity(
        row['sector_embeddings_x'].reshape(1, -1),
        row['sector_embeddings_y'].reshape(1, -1)
    )[0][0], axis=1)


## 📚 Step 4: Clean Names & Create Ground Truth and Precision@5 Evaluation
Standardize names in both the predicted and actual transaction datasets to allow comparison.


In [8]:
from collections import defaultdict

# === STEP 1: Clean names for matching ===
pairs_df['company_name_clean'] = pairs_df['company_name'].str.lower().str.strip()
pairs_df['pe_clean'] = pairs_df['PE'].str.lower().str.strip()
transactions_df['Target_clean'] = transactions_df['Target'].str.lower().str.strip()
transactions_df['PE_clean'] = transactions_df['PE'].str.lower().str.strip()

# === STEP 2: Create set of known PE–company matches ===
known_matches = set(zip(transactions_df['Target_clean'], transactions_df['PE_clean']))

# === STEP 3: Drop missing company names before grouping ===
valid_pairs = pairs_df.dropna(subset=['company_name'])

grouped = valid_pairs.groupby('company_name')
precision_at_k = defaultdict(list)

for company_name, group in grouped:
    group = group.sort_values('sector_similarity_score', ascending=False)
    top_k = group.head(5)

    company_name_clean = company_name.lower().strip()
    top_k_pe_clean = top_k['PE'].str.lower().str.strip()

    is_match = any((company_name_clean, pe) in known_matches for pe in top_k_pe_clean)
    precision_at_k[5].append(int(is_match))

# === STEP 4: Report Precision@5 if valid samples exist ===
if precision_at_k[5]:
    p_at_5 = sum(precision_at_k[5]) / len(precision_at_k[5])
    print(f"Precision@5 (cosine similarity): {p_at_5:.4f}")
else:
    print("No valid company–PE pairs were evaluated for Precision@5.")


No valid company–PE pairs were evaluated for Precision@5.


## 🧾 Step 5: Diagnostics
Check how many companies and PE firms are missing values, and how many theoretical matches exist in the dataset.


In [9]:
# Check for issues in the data used for Precision@5
print("Total pairs in dataframe:", len(pairs_df))
print("Company names missing:", pairs_df['company_name'].isnull().sum())
print("PE names missing:", pairs_df['PE'].isnull().sum())

# See how many cleaned pairs could theoretically match known transactions
pairs_df['pair_tuple'] = list(zip(pairs_df['company_name_clean'], pairs_df['pe_clean']))
match_count = pairs_df['pair_tuple'].isin(known_matches).sum()
print("Potential matches found in known transactions:", match_count)


Total pairs in dataframe: 0
Company names missing: 0
PE names missing: 0
Potential matches found in known transactions: 0


## 🔁 Step 6: Efficient Embedding with Progress Bar and Compute Cosine Similarity
Batch embedding both company and PE sectors for better performance.
Use `np.einsum` for fast similarity computation across all pairs.


In [10]:
company_df['key'] = 1
pe_df['key'] = 1

pairs_df = company_df.merge(pe_df, on='key').drop('key', axis=1)


In [11]:
print("New merged pairs_df shape:", pairs_df.shape)


New merged pairs_df shape: (319241, 56)


In [15]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

# Initialize SBERT
sbert = SentenceTransformer('all-MiniLM-L6-v2')

# Lowercase & fill NA
pairs_df['normalized_sectors_x'] = pairs_df['1º_characterization'].fillna('').str.lower()
pairs_df['normalized_sectors_y'] = pairs_df['Sectors'].fillna('').str.lower()

# Vectorized embeddings with progress bar
print("Encoding sector_emb_x...")
pairs_df['sector_emb_x'] = sbert.encode(
    pairs_df['normalized_sectors_x'].tolist(),
    show_progress_bar=True
).tolist()

print("Encoding sector_emb_y...")
pairs_df['sector_emb_y'] = sbert.encode(
    pairs_df['normalized_sectors_y'].tolist(),
    show_progress_bar=True
).tolist()

# Compute cosine similarity
print("Computing cosine similarity...")
company_matrix = np.vstack(pairs_df['sector_emb_x'].values)
pe_matrix = np.vstack(pairs_df['sector_emb_y'].values)

pairs_df['sector_similarity_score'] = np.einsum('ij,ij->i', company_matrix, pe_matrix) / (
    np.linalg.norm(company_matrix, axis=1) * np.linalg.norm(pe_matrix, axis=1)
)


Encoding sector_emb_x...


Batches: 100%|██████████| 9977/9977 [03:32<00:00, 46.87it/s]


Encoding sector_emb_y...


Batches: 100%|██████████| 9977/9977 [09:43<00:00, 17.11it/s]


Computing cosine similarity...


In [16]:
import numpy as np

# Convert list of vectors to arrays
company_matrix = np.stack(pairs_df['sector_emb_x'].values)
pe_matrix = np.stack(pairs_df['sector_emb_y'].values)

# Efficient cosine similarity calculation
pairs_df['sector_similarity_score'] = np.einsum('ij,ij->i', company_matrix, pe_matrix) / (
    np.linalg.norm(company_matrix, axis=1) * np.linalg.norm(pe_matrix, axis=1)
)


## ✅ Step 7: Final Precision@5 Evaluation
Final ranking evaluation after optimization.


In [17]:
from collections import defaultdict

# Group by company and rank PEs
grouped = pairs_df.groupby('company_name_clean')
precision_at_k = defaultdict(list)

for company_name, group in grouped:
    top_k = group.sort_values('sector_similarity_score', ascending=False).head(5)
    top_k_pe_clean = top_k['pe_clean']

    is_match = any((company_name, pe) in known_matches for pe in top_k_pe_clean)
    precision_at_k[5].append(int(is_match))

# Report Precision@5
if len(precision_at_k[5]) > 0:
    p_at_5 = sum(precision_at_k[5]) / len(precision_at_k[5])
    print(f"Precision@5 (cosine similarity): {p_at_5:.4f}")
else:
    print("No valid company–PE pairs were evaluated for Precision@5.")


Precision@5 (cosine similarity): 0.0000
